# Azure AI Language Services Demo

This notebook demonstrates the capabilities of Azure AI Language Services including:
- Sentiment Analysis
- Key Phrase Extraction
- Language Detection
- Named Entity Recognition
- Text Classification

## Prerequisites
- Azure subscription
- Azure AI Language resource created in Azure portal
- Python 3.8 or higher

## 1. Setup and Package Installation

First, let's install the required Azure SDK packages for Language services.

In [10]:
# Install required Azure SDK packages
!pip install azure-ai-textanalytics azure-identity azure-core python-dotenv


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# Import required libraries
import os
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
import json
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Azure Authentication and Configuration

### Option 1: Using Environment Variables (Recommended)
Set these environment variables in your system or create a `.env` file:
```
AZURE_LANGUAGE_ENDPOINT=https://your-resource.cognitiveservices.azure.com/
AZURE_LANGUAGE_KEY=your-api-key
```

### Option 2: Using Managed Identity (For Azure-hosted applications)
When running on Azure services like App Service, Container Apps, or Azure VMs with managed identity enabled.

In [12]:
# Configuration for Azure AI Language Service
# Replace these with your actual values or use environment variables

# Option 1: Using API Key (for development/testing)
endpoint = os.getenv('AZURE_LANGUAGE_ENDPOINT') or "https://your-resource.cognitiveservices.azure.com/"
api_key = os.getenv('AZURE_LANGUAGE_KEY') or "your-api-key-here"

# Create client using API key
if api_key and api_key != "your-api-key-here":
    credential = AzureKeyCredential(api_key)
    client = TextAnalyticsClient(endpoint=endpoint, credential=credential)
    print("✅ Client created using API Key authentication")
else:
    # Option 2: Using Managed Identity (recommended for production)
    try:
        credential = DefaultAzureCredential()
        client = TextAnalyticsClient(endpoint=endpoint, credential=credential)
        print("✅ Client created using Managed Identity authentication")
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        print("Please set AZURE_LANGUAGE_ENDPOINT and AZURE_LANGUAGE_KEY environment variables")

✅ Client created using API Key authentication


## 3. Sentiment Analysis

Sentiment analysis determines the emotional tone of text content. It returns:
- **Positive**: Happy, satisfied, pleased
- **Negative**: Sad, angry, disappointed
- **Neutral**: Factual, objective statements
- **Mixed**: Contains both positive and negative sentiments

In [13]:
def analyze_sentiment(texts):
    """
    Analyze sentiment of given texts
    """
    try:
        response = client.analyze_sentiment(documents=texts, show_opinion_mining=True)
        
        for idx, doc in enumerate(response):
            if doc.is_error:
                print(f"Document {idx}: Error - {doc.error}")
            else:
                print(f"\n📝 Text: '{texts[idx]}'")
                print(f"🎯 Overall Sentiment: {doc.sentiment}")
                print(f"📊 Confidence Scores:")
                print(f"   • Positive: {doc.confidence_scores.positive:.2f}")
                print(f"   • Negative: {doc.confidence_scores.negative:.2f}")
                print(f"   • Neutral: {doc.confidence_scores.neutral:.2f}")
                
                # Show sentence-level sentiment
                if doc.sentences:
                    print(f"📄 Sentence-level analysis:")
                    for i, sentence in enumerate(doc.sentences):
                        print(f"   Sentence {i+1}: {sentence.sentiment} (confidence: {sentence.confidence_scores.positive:.2f})")
                        
    except Exception as e:
        print(f"❌ Error analyzing sentiment: {e}")

# Test sentiment analysis with sample texts
sample_texts = [
    "I love using Azure AI services! They are amazing and very powerful.",
    "The weather is terrible today. I'm so disappointed.",
    "Azure AI Language service provides text analytics capabilities.",
    "The product is okay, but the customer service was excellent!"
]

print("🔍 Analyzing sentiment for sample texts...")
analyze_sentiment(sample_texts)

🔍 Analyzing sentiment for sample texts...

📝 Text: 'I love using Azure AI services! They are amazing and very powerful.'
🎯 Overall Sentiment: positive
📊 Confidence Scores:
   • Positive: 1.00
   • Negative: 0.00
   • Neutral: 0.00
📄 Sentence-level analysis:
   Sentence 1: positive (confidence: 1.00)
   Sentence 2: positive (confidence: 1.00)

📝 Text: 'The weather is terrible today. I'm so disappointed.'
🎯 Overall Sentiment: negative
📊 Confidence Scores:
   • Positive: 0.00
   • Negative: 0.97
   • Neutral: 0.02
📄 Sentence-level analysis:
   Sentence 1: negative (confidence: 0.00)
   Sentence 2: negative (confidence: 0.00)

📝 Text: 'Azure AI Language service provides text analytics capabilities.'
🎯 Overall Sentiment: neutral
📊 Confidence Scores:
   • Positive: 0.00
   • Negative: 0.00
   • Neutral: 1.00
📄 Sentence-level analysis:
   Sentence 1: neutral (confidence: 0.00)

📝 Text: 'The product is okay, but the customer service was excellent!'
🎯 Overall Sentiment: positive
📊 Confidence Sc

## 4. Key Phrase Extraction

Key phrase extraction identifies the main talking points in text content. This is useful for:
- Summarizing content
- Creating tags or keywords
- Understanding main topics

In [14]:
def extract_key_phrases(texts):
    """
    Extract key phrases from given texts
    """
    try:
        response = client.extract_key_phrases(documents=texts)
        
        for idx, doc in enumerate(response):
            if doc.is_error:
                print(f"Document {idx}: Error - {doc.error}")
            else:
                print(f"\n📝 Text: '{texts[idx]}'")
                print(f"🔑 Key Phrases:")
                for phrase in doc.key_phrases:
                    print(f"   • {phrase}")
                    
    except Exception as e:
        print(f"❌ Error extracting key phrases: {e}")

# Test key phrase extraction
key_phrase_texts = [
    "Microsoft Azure is a cloud computing platform and infrastructure created by Microsoft for building, testing, deploying, and managing applications and services through Microsoft-managed data centers.",
    "Artificial Intelligence and Machine Learning are transforming how businesses operate in the modern digital economy.",
    "The new electric vehicle features advanced battery technology, autonomous driving capabilities, and sustainable materials."
]

print("🔍 Extracting key phrases from sample texts...")
extract_key_phrases(key_phrase_texts)

🔍 Extracting key phrases from sample texts...

📝 Text: 'Microsoft Azure is a cloud computing platform and infrastructure created by Microsoft for building, testing, deploying, and managing applications and services through Microsoft-managed data centers.'
🔑 Key Phrases:
   • cloud computing platform
   • Microsoft-managed data centers
   • Microsoft Azure
   • infrastructure
   • testing
   • applications
   • services

📝 Text: 'Artificial Intelligence and Machine Learning are transforming how businesses operate in the modern digital economy.'
🔑 Key Phrases:
   • modern digital economy
   • Artificial Intelligence
   • Machine Learning
   • businesses

📝 Text: 'The new electric vehicle features advanced battery technology, autonomous driving capabilities, and sustainable materials.'
🔑 Key Phrases:
   • new electric vehicle
   • advanced battery technology
   • autonomous driving capabilities
   • sustainable materials

📝 Text: 'Microsoft Azure is a cloud computing platform and infrastr

## 5. Language Detection

Language detection identifies the language of text content. This is useful for:
- Content routing and localization
- Translation workflows
- Multi-language content management

In [15]:
def detect_language(texts):
    """
    Detect the language of given texts
    """
    try:
        response = client.detect_language(documents=texts)
        
        for idx, doc in enumerate(response):
            if doc.is_error:
                print(f"Document {idx}: Error - {doc.error}")
            else:
                print(f"\n📝 Text: '{texts[idx]}'")
                print(f"🌍 Primary Language: {doc.primary_language.name} ({doc.primary_language.iso6391_name})")
                print(f"📊 Confidence Score: {doc.primary_language.confidence_score:.2f}")
                        
    except Exception as e:
        print(f"❌ Error detecting language: {e}")

# Test language detection with multilingual texts
multilingual_texts = [
    "Hello, how are you today?",
    "Bonjour, comment allez-vous aujourd'hui?",
    "Hola, ¿cómo estás hoy?",
    "Guten Tag, wie geht es dir heute?",
    "こんにちは、今日はいかがですか？",
    "Привет, как дела сегодня?"
]

print("🔍 Detecting languages for sample texts...")
detect_language(multilingual_texts)

🔍 Detecting languages for sample texts...

📝 Text: 'Hello, how are you today?'
🌍 Primary Language: English (en)
📊 Confidence Score: 0.82

📝 Text: 'Bonjour, comment allez-vous aujourd'hui?'
🌍 Primary Language: French (fr)
📊 Confidence Score: 1.00

📝 Text: 'Hola, ¿cómo estás hoy?'
🌍 Primary Language: Spanish (es)
📊 Confidence Score: 1.00

📝 Text: 'Guten Tag, wie geht es dir heute?'
🌍 Primary Language: German (de)
📊 Confidence Score: 1.00

📝 Text: 'こんにちは、今日はいかがですか？'
🌍 Primary Language: Japanese (ja)
📊 Confidence Score: 1.00

📝 Text: 'Привет, как дела сегодня?'
🌍 Primary Language: Russian (ru)
📊 Confidence Score: 1.00

📝 Text: 'Hello, how are you today?'
🌍 Primary Language: English (en)
📊 Confidence Score: 0.82

📝 Text: 'Bonjour, comment allez-vous aujourd'hui?'
🌍 Primary Language: French (fr)
📊 Confidence Score: 1.00

📝 Text: 'Hola, ¿cómo estás hoy?'
🌍 Primary Language: Spanish (es)
📊 Confidence Score: 1.00

📝 Text: 'Guten Tag, wie geht es dir heute?'
🌍 Primary Language: German (de)
📊 Con

## 6. Named Entity Recognition (NER)

Named Entity Recognition identifies and categorizes named entities in text such as:
- **Person**: Names of people
- **Location**: Geographic locations
- **Organization**: Company names, institutions
- **DateTime**: Dates and times
- **Quantity**: Numbers and measurements
- **Email**: Email addresses
- **URL**: Web addresses

In [16]:
def recognize_entities(texts):
    """
    Recognize named entities in given texts
    """
    try:
        response = client.recognize_entities(documents=texts)
        
        for idx, doc in enumerate(response):
            if doc.is_error:
                print(f"Document {idx}: Error - {doc.error}")
            else:
                print(f"\n📝 Text: '{texts[idx]}'")
                print(f"🏷️ Named Entities:")
                
                if doc.entities:
                    # Group entities by category
                    entities_by_category = {}
                    for entity in doc.entities:
                        category = entity.category
                        if category not in entities_by_category:
                            entities_by_category[category] = []
                        entities_by_category[category].append({
                            'text': entity.text,
                            'confidence': entity.confidence_score
                        })
                    
                    # Display entities by category
                    for category, entities in entities_by_category.items():
                        print(f"   📂 {category}:")
                        for entity in entities:
                            print(f"      • {entity['text']} (confidence: {entity['confidence']:.2f})")
                else:
                    print(f"   No entities found.")
                    
    except Exception as e:
        print(f"❌ Error recognizing entities: {e}")

# Test named entity recognition
entity_texts = [
    "Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975, in Albuquerque, New Mexico.",
    "Please contact John Smith at john.smith@company.com or visit our website at https://www.example.com for more information.",
    "The meeting is scheduled for December 15, 2024, at 2:30 PM in the Seattle office.",
    "Apple Inc. reported revenue of $394.3 billion in fiscal year 2022."
]

print("🔍 Recognizing named entities in sample texts...")
recognize_entities(entity_texts)

🔍 Recognizing named entities in sample texts...

📝 Text: 'Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975, in Albuquerque, New Mexico.'
🏷️ Named Entities:
   📂 Organization:
      • Microsoft (confidence: 0.99)
   📂 Person:
      • Bill Gates (confidence: 1.00)
      • Paul Allen (confidence: 1.00)
   📂 DateTime:
      • April 4, 1975 (confidence: 1.00)
   📂 Location:
      • Albuquerque (confidence: 1.00)
      • New Mexico (confidence: 0.97)

📝 Text: 'Please contact John Smith at john.smith@company.com or visit our website at https://www.example.com for more information.'
🏷️ Named Entities:
   📂 Person:
      • John Smith (confidence: 1.00)
   📂 Email:
      • john.smith@company.com (confidence: 0.80)
   📂 URL:
      • https://www.example.com (confidence: 0.80)

📝 Text: 'The meeting is scheduled for December 15, 2024, at 2:30 PM in the Seattle office.'
🏷️ Named Entities:
   📂 Event:
      • meeting (confidence: 0.59)
   📂 DateTime:
      • December 15, 2024, at 2:

## 7. Combined Analysis Example

Let's combine multiple language analysis features to get comprehensive insights from a single text.

In [17]:
def comprehensive_text_analysis(text):
    """
    Perform comprehensive analysis on a single text
    """
    print(f"🔍 Comprehensive Analysis for: '{text}'")
    print("=" * 80)
    
    # Language Detection
    print("\n🌍 LANGUAGE DETECTION:")
    detect_language([text])
    
    # Sentiment Analysis
    print("\n😊 SENTIMENT ANALYSIS:")
    analyze_sentiment([text])
    
    # Key Phrase Extraction
    print("\n🔑 KEY PHRASE EXTRACTION:")
    extract_key_phrases([text])
    
    # Named Entity Recognition
    print("\n🏷️ NAMED ENTITY RECOGNITION:")
    recognize_entities([text])
    
    print("\n" + "=" * 80)

# Example comprehensive analysis
sample_text = "Microsoft Azure AI services have revolutionized how businesses process and understand text data. Founded in Seattle, these cloud-based solutions offer incredible accuracy and performance. Contact our team at support@azure.com to learn more about implementing these technologies in your organization by January 2025."

comprehensive_text_analysis(sample_text)

🔍 Comprehensive Analysis for: 'Microsoft Azure AI services have revolutionized how businesses process and understand text data. Founded in Seattle, these cloud-based solutions offer incredible accuracy and performance. Contact our team at support@azure.com to learn more about implementing these technologies in your organization by January 2025.'

🌍 LANGUAGE DETECTION:

📝 Text: 'Microsoft Azure AI services have revolutionized how businesses process and understand text data. Founded in Seattle, these cloud-based solutions offer incredible accuracy and performance. Contact our team at support@azure.com to learn more about implementing these technologies in your organization by January 2025.'
🌍 Primary Language: English (en)
📊 Confidence Score: 0.98

😊 SENTIMENT ANALYSIS:

📝 Text: 'Microsoft Azure AI services have revolutionized how businesses process and understand text data. Founded in Seattle, these cloud-based solutions offer incredible accuracy and performance. Contact our team at sup

## 8. Interactive Text Analysis

Try analyzing your own text! Replace the text below with your content.

In [18]:
# 🎯 Try your own text analysis!
# Replace the text below with your own content

your_text = "Enter your text here for analysis..."

# Uncomment the line below to run analysis on your text
# comprehensive_text_analysis(your_text)

print("📝 Replace 'your_text' variable with your content and uncomment the analysis line to test!")

📝 Replace 'your_text' variable with your content and uncomment the analysis line to test!


## 9. Best Practices and Tips

### 🔒 Security Best Practices
1. **Never hardcode API keys** in your code
2. **Use environment variables** or Azure Key Vault for secrets
3. **Use Managed Identity** when running on Azure services
4. **Implement proper error handling** and retry logic
5. **Monitor API usage** to avoid unexpected costs

### 📊 Performance Tips
1. **Batch requests** when analyzing multiple documents
2. **Use connection pooling** for high-volume applications
3. **Implement caching** for frequently analyzed content
4. **Monitor rate limits** and implement backoff strategies

### 💡 Usage Guidelines
1. **Text length limits**: Up to 5,120 characters per document
2. **Batch size limits**: Up to 10 documents per request
3. **Rate limits**: Vary by pricing tier
4. **Language support**: 100+ languages for detection, fewer for other features

### 🔧 Troubleshooting
- Check endpoint URL format
- Verify API key permissions
- Ensure proper authentication method
- Monitor quota and billing limits

## 10. Conclusion and Next Steps

🎉 **Congratulations!** You've successfully explored Azure AI Language Services capabilities including:
- ✅ Sentiment Analysis
- ✅ Key Phrase Extraction  
- ✅ Language Detection
- ✅ Named Entity Recognition

### 🚀 Next Steps
1. **Explore other notebooks** in this series:
   - Azure AI Speech Services
   - Azure AI Vision Services
   - Azure AI Document Intelligence

2. **Build real applications** using these services:
   - Content moderation systems
   - Customer feedback analysis
   - Multi-language content management
   - Document processing pipelines

3. **Learn more** about Azure AI:
   - [Azure AI Language Documentation](https://docs.microsoft.com/azure/cognitive-services/language-service/)
   - [Azure AI SDK Samples](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/textanalytics)
   - [Azure AI Pricing](https://azure.microsoft.com/pricing/details/cognitive-services/)

### 📚 Additional Resources
- [Azure AI Language Service Overview](https://azure.microsoft.com/products/ai-services/ai-language/)
- [Python SDK Reference](https://docs.microsoft.com/python/api/azure-ai-textanalytics/)
- [REST API Reference](https://docs.microsoft.com/rest/api/language/)